In [262]:
from sklearn.datasets import load_digits
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
data=load_digits()

In [218]:
data['target_names']

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [3]:
X=pd.DataFrame(data["data"])

In [5]:
Y=pd.Series(data["target"])

In [242]:
Y_b=label_binarize(Y,classes=list(range(10)))

In [252]:
X_test, X_train, y_test, y_train=train_test_split(X, Y, test_size=0.2, random_state=666 )

In [254]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

##### Набираем модели: возьмём, логРегрессию, Деревья и Соседей

In [255]:
lr=LogisticRegression
dtc=DecisionTreeClassifier
knn=KNeighborsClassifier

In [230]:
models={'lr':LogisticRegression,'dtc':DecisionTreeClassifier, 'knn':KNeighborsClassifier}

##### Набираем параметры для моделей

In [264]:
cs=10**np.linspace(-3,1,5)
grid1={"C":cs, "penalty":['l1','l2'],'solver':['liblinear'], 'dual':[bool(0)]}
gs1=GridSearchCV(lr(),grid1,scoring='accuracy', cv=5)
gs1.fit(X_train,y_train)
gs1.best_params_
#model1=lr(**gs1.best_params_)
#model1.fit(X_train,y_train)
#pred1=model1.predict(X_test)
#accuracy_score(y_test,pred1)

{'C': 0.01, 'dual': False, 'penalty': 'l2', 'solver': 'liblinear'}

In [89]:
grid2={"max_depth":list(range(1,20)), 'criterion':['gini','entropy'], 'splitter':['best','random'], 'class_weight':['balanced',None]}
gs2=GridSearchCV(DecisionTreeClassifier(), grid2, scoring='accuracy')
gs2.fit(X_train, y_train)
gs2.best_params_

{'class_weight': None,
 'criterion': 'entropy',
 'max_depth': 8,
 'splitter': 'best'}

In [26]:
grid3={'n_neighbors':list(range(1,6)), 'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute'], 'weights':['uniform','distance'], 'p':[1,2]}
gs3=GridSearchCV(knn(),grid3, scoring='accuracy')
gs3.fit(X_train,y_train)
gs3.best_params_
#model3=knn(**gs3.best_params_)
#model3.fit(X_train,y_train)
#pred3=model3.predict(X_test)
#accuracy_score(y_test, pred3)

{'algorithm': 'auto', 'n_neighbors': 1, 'p': 2, 'weights': 'uniform'}

##### Запихиваем в цикл

In [225]:
score=[]
for x in models.values():
    if x==lr:
        grids=grid1
    elif x==dtc:
        grids=grid2
    elif x==knn:
        grids=grid3
    gs=GridSearchCV(x(), grids, scoring='accuracy')
    gs.fit(X_train,y_train)
    model=x(**gs.best_params_)
    model.fit(X_train,y_train)
    pred=model.predict(X_test)
    pred_pr=model.predict_proba(X_test)
    score.append(accuracy_score(y_test,pred ))
print(*(zip(score, models.keys())))

(0.94084899095337504, 'lr') (0.7668754349338901, 'dtc') (0.96520528879610301, 'knn')


Получаем, что лучшая модель у нас **KNN**, набором параметров: ** {'algorithm': 'auto', 'n_neighbors': 1, 'p': 2, 'weights': 'uniform'}**